# Notes: `Understanding Python Metaclasses`

Em python tudo é um objeto (tudo mesmo) até mesmo as classes criadas para criar novas instâncias são objetos. As classes então, são instâncias de algo chamado `metaclass`. Em python a metaclass default da quais as classes são instâncias é a `type`. Isso é um pouco confuso pois type também é a built-in function que retorna o tipo de um objeto.

In [1]:
class Student:
    pass

In [2]:
# Aqui podemos ver que a classe em si é uma instância da metaclass type

type(Student)

type

In [5]:
# E analisando a instância da classe Student, a função type retorna que o tipo é o __main__.Student

student = Student()

type(student)

__main__.Student

Ou da mesma forma podemos fazer a análise utilizando a função built-in `isinstance`

In [7]:
print(isinstance(Student, type))
print(isinstance(student, Student))

True
True


Verifico assim a existência de uma metaclasse responsável por criar as classes em sí.

# Diagrama de criação de instâncias e classes

![](./src/diagram_instances_class_metaclass.jpg)

### Criando uma classe a partir do built-in type.

In [9]:
MyClass = type('MyClass', (), {})
MyClass

__main__.MyClass

Reparo que consigo criar uma classe sem utilizar a keyword `class`.

Já tinha visto em outras fontes essa forma de criação de classes.

- Livro sobre design patterns (falando sobre singleton)
- Outros artigos. 

Nesta forma de criação, a tuple posicionada no segundo argumento cuida da questão hierarquica, definindo de quais classes a classe criada irá herdar. O dicionário presente no terceiro argumento representa a definição dos atributos.

In [11]:
# Outro exemplo

School = type('School', tuple([object]), {'name': 'Colégio Mercúrio', 'address': 'Rua Mercúrio, 179'})

In [18]:
School.__base__

object

In [19]:
print(School.name)
print(School.address)

Colégio Mercúrio
Rua Mercúrio, 179


### Questionamento:

quando passo os atributos dessa forma através da built-in type, eu na verdade estou criando atributos da classe compartilhado entre as instâncias? `Boa pergunta!`

# Criando uma `metaclass` customizada:

In [39]:
class Meta(type):
    
    def __call__(self):
        
        print("teste chamada de metaclasse")

class Complex(metaclass=Meta):
    
    pass

print(type(Complex))

Complex()

<class '__main__.Meta'>
teste chamada de metaclasse


Consigo então, criar uma <font color='green'>metaclass</font> customizada baseada no tipo default `type` e posso utiliza-lo para ser a <font color='green'>metaclass</font> de refência para instância de outras classes.
Outro teste que lembrei de fazer foi observar que a criação de uma instância de uma classe (objeto convencional), chama o `__call__` da metaclass. Com isso consigo entender porque esta pode ser um método utilizado em `singleton`, colocando como responsabilidade de uma <font color='green'>metaclass</font> o controle sobre o número de instâncias criadas. 

O artigo comenta que a palavra-chave <font color='blue'>class</font> não é somente uma abstração de linguagem para facilitar a utilização (syntatic sugar) mas que é mais que isso, a chamada dessa forma faz algumas coisas extras como chamar alguns dunder methods que ajudam na criação e definição da classe.

# Magical methods

Neste tópico é comentado como o python utiliza os dunder methods como `syntatic sugar` para várias operação. 

- É dado o exemplo como `__call__`
- Vários outros exemplos:
    - __enter__ e __exit__ para fazer o context manager
    - __eq__ e outros para tornar os objetos ordenáveis
    - __add__ para tornar os objetos somáveis

- e por ai vai

In [20]:
class Funky():
    
    def __call__(self):
        
        print('Look at me, I work like a function')
        
f = Funky()

# com a definição do método call, o objeto pode ser chamado como uma função:

f()

Look at me, I work like a function


In [21]:
import sys

sys.getsizeof(f)

48

In [22]:
', '.join(dir(f))

'__call__, __class__, __delattr__, __dict__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __gt__, __hash__, __init__, __init_subclass__, __le__, __lt__, __module__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __str__, __subclasshook__, __weakref__'

In [28]:
len(dir(f))

27

In [34]:
f.__dict__

{}

# Utilizando __slots__

o `__slots__` é um atributo de classe que é utilizado para definir quais atributos e métodos de fato a classe irá ter. Evita de reservar espaço para dunder methods e outros atributos. Promove uma economia de espaço. Resta avaliar quando utilizar isso faz sentido mas valeu o aprendizado. Não permite a criação de um dicionário contendo os atributos.

In [29]:
class FunkyWithSlots():
    
    __slots__ = ()
    
    def __call__(self):
        
        print('Look at me, I work like a function')
        
f_slots = FunkyWithSlots()

In [30]:
sys.getsizeof(f_slots)

32

In [31]:
', '.join(dir(f_slots))

'__call__, __class__, __delattr__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __gt__, __hash__, __init__, __init_subclass__, __le__, __lt__, __module__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __slots__, __str__, __subclasshook__'

In [32]:
len(dir(f_slots))

26

# Utilização de dunder methods

In [54]:
class Magic():
    
    @property
    def __repr__(self):
        
        def inner():
            
            return "Hi! I'm here!"
        
        return inner
    
# Qual a diferença entre a implementação acima e a implementação abaixo?

class Magic2():
    
    def __repr__(self):
        
        return "Hi! I'm here!"

In [55]:
magic = Magic()
magic_2 = Magic2()

In [52]:
repr(magic)

"Hi! I'm here!"

In [56]:
repr(magic_2)

"Hi! I'm here!"

Pensar um pouco melhor nessas implementações não entendi muito bem a diferença entre elas. 

# O método `__new__`

O método `__new__` é o construtor (retorna uma nova instância) enquanto que o `__init__` é somente um inicializador (a instância já está criada quando o `__init__` é chamado).

In [75]:
class Foobar:
    def __new__(cls):
        print('Inside new method')
        
        instance = super().__new__(cls)
        instance.class_attr = 'class_attr'
        
        return instance
    
    def __init__(self):
        
        self.name = 'rodrigo'
        self.surname = 'bernardo'

In [76]:
foobar = Foobar()

Inside new method


Agora, se lembrarmos da seção anterior, esperamos que o `__new__` fosse procurado na `metaclass`, mas dessa forma este método não seria tão útil, então ele é procurado estaticamente.
Quando a classe Foobar quiser este método mágico, ele será procurado no mesmo objeto (a classe), e não no nível acima como nos outros métodos mágicos. Este ponto é muito importante ser entendido, pois ambas as classe e metaclasse podem definir este método:

Foobar.__new__ is used to create instances of Foobar
type.__new__ is used to create the Foobar class (an instance of type in the example)

- `Foobar.__new__` é utilizado para criar uma instancia de Foobar
- `type.__new__` é usado para criar a Foobar class (uma instância de `type` neste exemplo)

# O método `__prepare__`

In [100]:
class PrepareTest():
    
    x = 1
    y = 2
    z = 3

In [101]:
prepare_test = PrepareTest()

{}


In [88]:
PrepareTest.__prepare__()

{}

In [83]:
type.__prepare__()

{}

In [85]:
help(type.__prepare__)

Help on built-in function __prepare__:

__prepare__(...) method of builtins.type instance
    __prepare__() -> dict
    used to create the namespace for the class statement



O `__prepare__` é um método da metaclasse, mas não é um método da instância da classe. *São coisas diferentes?* Ou seja, este método não é implementado na instância embora exista na classe em si (quando enxergada como uma instância da `metaclass type` por exemplo). Começo a perceber que a classe em si e suas instâncias são coisas diferentes. 

# Aprofundando um pouco mais em `Metaclass`

In [102]:
class Foo(metaclass=print):  # pointless, but illustrative
    pass

Foo () {'__module__': '__main__', '__qualname__': 'Foo'}


Se uma função for utilizada como metaclass ela herda o tipo que a função retorna

In [109]:
def retorna_dict(*args, **kwargs):
    return dict()

class Bar(metaclass=retorna_dict):  # pointless, but illustrative
    pass